# Data cleaning

By the end of the tutorials of this week, you should acquire:

**Knowledge on:**
* Inspection of dataframes
* Identification and handling of missing values
* Usage of functions to clean columns
* Merging dataframes
* Data minimization
* Basic privacy preserving measures

**Skills:**
* .describe(), .dtypes
* .isna().sum(), .fillna(), . dropna()
* .apply() and selection of functions based on existing list
* .merge()
* .sample()

In [1]:
import pandas as pd

## Loading data

Instead of using Twitter data for this tutorial, I decided to use data from YouTube. I collected it using the [YouTube Data Tools](https://tools.digitalmethods.net/netvizz/youtube/), also created by the Digital Methods Initiative from the UvA.

The data I have is from a video search using the keyword "climate change".

It always helps if we note down what we want to do with the data *before* we start. We can always refine these objectives later.

### What do I want to achieve with this analysis?

Ideally I have a few research questions here. While we won't be able to do the visualisations (which we will learn on week 4) or statistical testing (week 5 and 6), I am noting down some research questions that may be interesting.

* RQ1. To what extent does the sentiment expressed in the title of the video influence user engagement (views, likes and dislikes)?
* RQ2. To what extent does the sentiment expressed in the title of the video vary depending on the category in which the video is published?

**Important:** I only want to do this for videos published in 2021. 

These are examples - which are probably not very sophisticated yet as a business challenge - but they imply that we need a few things:
1. We need to have a sentiment analysis performed in the titles of the videos so we have the **sentiment** variable(s)
2. I need to make sure I have the user engagement variables (**likes**, **dislikes**, and **views**)
3. I need to have a variable for the **category** of the video
4. I need to know **when the video was published**, and remove old videos

With this noted down, I can start loading, inspecting and cleaning the data.

In [2]:
videos = pd.read_csv('videolist_search500_2021_09_16-08_37_30.tab')

ParserError: Error tokenizing data. C error: Expected 22 fields in line 3, saw 158


Because this is a tab-delimited file (i.e., the separators are tabs, not commas), I need to specify this in the read_csv command.

In [3]:
videos = pd.read_csv('videolist_search500_2021_09_16-08_37_30.tab', sep='\t')

In [4]:
len(videos)

500

In [5]:
videos.head()

,position,channelId,channelTitle,videoId,publishedAt,publishedAtSQL,videoTitle,videoDescription,tags,videoCategoryId,...,dimension,definition,caption,thumbnail_maxres,licensedContent,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,1,UC18vz5hUUqxbGvym9ghtX_w,Full Frontal with Samantha Bee,xkgt1Avnvw0,2021-09-16T05:59:45Z,2021-09-16 05:59:45,Vivian Howard: The Perfect Recipe for Combatin...,Apocalyptic climate crisis got you down? Allow...,"Full Frontal with Samantha Bee,Full Frontal,Sa...",24,...,2d,hd,False,https://i.ytimg.com/vi/xkgt1Avnvw0/maxresdefau...,1.0,796.0,88.0,2.0,0,9.0
1,2,UCJ6DCjlsOB8dwCrDbOLia6g,Info Viral,Sf-klthGrqA,2021-09-16T06:00:06Z,2021-09-16 06:00:06,"Global climate change!Cameroon under the snow,...",#GlobalClimateChange #CameroonUnderTheSnow #Sn...,"snow,snow fall,snow falling,snow fall in Camer...",26,...,2d,hd,False,NaN,1.0,2.0,0.0,0.0,0,0.0
2,3,UC18vz5hUUqxbGvym9ghtX_w,Full Frontal with Samantha Bee,4Z0O1lJBU7g,2021-09-16T05:59:46Z,2021-09-16 05:59:46,Food Waste and Climate Change: How Your Leftov...,The recent devastating climate report left man...,"Full Frontal with Samantha Bee,Full Frontal,Sa...",24,...,2d,hd,False,https://i.ytimg.com/vi/4Z0O1lJBU7g/maxresdefau...,1.0,1735.0,152.0,5.0,0,22.0
3,4,UCEfvFsy9qbzeKyASsDs0V-w,KJ Singh,KWoI9jTHHlk,2021-09-16T05:38:04Z,2021-09-16 05:38:04,Climate Change in Australia and Brexit #downun...,NaN,NaN,22,...,2d,hd,False,https://i.ytimg.com/vi/KWoI9jTHHlk/maxresdefau...,NaN,0.0,0.0,0.0,0,0.0
4,5,UC7pluR6rB5KZIbN2IxamzxQ,BBC News Marathi,32y2dG2tLOg,2021-09-16T04:58:32Z,2021-09-16 04:58:32,Climate Change : 50c सेल्शिअस तापमानाच्या ठिका...,#ClimateChange #Temperature #Heat नायजेरिया दे...,"Global Warming,Warming,Hottest place in world,...",25,...,2d,hd,False,NaN,1.0,5442.0,81.0,3.0,0,5.0


In [6]:
videos.columns

Index(['position', 'channelId', 'channelTitle', 'videoId', 'publishedAt',
       'publishedAtSQL', 'videoTitle', 'videoDescription', 'tags',
       'videoCategoryId', 'videoCategoryLabel', 'duration', 'durationSec',
       'dimension', 'definition', 'caption', 'thumbnail_maxres',
       'licensedContent', 'viewCount', 'likeCount', 'dislikeCount',
       'favoriteCount', 'commentCount'],
      dtype='object')

In [7]:
videos.describe()

,position,videoCategoryId,durationSec,licensedContent,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
count,500.000000,500.000000,500.000000,386.0,4.990000e+02,489.000000,489.000000,500.0,460.000000
mean,250.500000,24.896000,686.952000,1.0,3.573237e+05,9466.768916,881.370143,0.0,1588.645652
std,144.481833,3.237615,749.238875,0.0,1.253590e+06,40750.021842,5405.282802,0.0,5599.554476
min,1.000000,1.000000,15.000000,1.0,0.000000e+00,0.000000,0.000000,0.0,0.000000
25%,125.750000,25.000000,231.500000,1.0,3.392500e+03,80.000000,10.000000,0.0,27.750000
50%,250.500000,25.000000,427.000000,1.0,2.449100e+04,438.000000,58.000000,0.0,199.500000
75%,375.250000,27.000000,778.250000,1.0,1.197805e+05,2969.000000,236.000000,0.0,787.500000
max,500.000000,30.000000,3585.000000,1.0,1.534203e+07,478296.000000,91967.000000,0.0,68194.000000


In [9]:
videos.isna().sum()

position                0
channelId               0
channelTitle            0
videoId                 0
publishedAt             0
publishedAtSQL          0
videoTitle              0
videoDescription       13
tags                   55
videoCategoryId         0
videoCategoryLabel      0
duration                0
durationSec             0
dimension               0
definition              0
caption                 0
thumbnail_maxres      117
licensedContent       114
viewCount               1
likeCount              11
dislikeCount           11
favoriteCount           0
commentCount           40
dtype: int64

*Some preliminary findings:*
1. It seems I have the variables measuring engagement I need, but likeCount and dislikeCount seem to have missing values
2. I need to run sentiment analysis on videoTitle, first I need to figure out what the language is of the titles (as this seems not to be provided by the API)
3. The videoCategoryLabel column seems to be a starting point for the category variable.
4. The publishedAt column can probably help me filter videos from 2021.

All of this still needs to be confirmed though...

# Data cleaning

Now let's start preparing the data. The steps always depend on the dataset, but at a minimum we need to make sure that we:
1. Handle the missing values for relevant variables
2. Check if the variables are stored in the correct format/type
3. Create the variables we need (that may be not in the data yet)

### Missing values

In [10]:
videos.isna().sum()

position                0
channelId               0
channelTitle            0
videoId                 0
publishedAt             0
publishedAtSQL          0
videoTitle              0
videoDescription       13
tags                   55
videoCategoryId         0
videoCategoryLabel      0
duration                0
durationSec             0
dimension               0
definition              0
caption                 0
thumbnail_maxres      117
licensedContent       114
viewCount               1
likeCount              11
dislikeCount           11
favoriteCount           0
commentCount           40
dtype: int64

From the variables I am interested in, seems like likeCount, dislikeCount and viewCount are the ones that do have an issue. Let me check what's happening with them.

One possibility would be that 0's are not included (i.e., if the video does not have a like, it will not appear). Let's see if that's the case...

In [11]:
videos[['likeCount', 'dislikeCount', 'viewCount']].describe()

,likeCount,dislikeCount,viewCount
count,489.000000,489.000000,4.990000e+02
mean,9466.768916,881.370143,3.573237e+05
std,40750.021842,5405.282802,1.253590e+06
min,0.000000,0.000000,0.000000e+00
25%,80.000000,10.000000,3.392500e+03
50%,438.000000,58.000000,2.449100e+04
75%,2969.000000,236.000000,1.197805e+05
max,478296.000000,91967.000000,1.534203e+07


The minimum value is 0, so probably something else is going on. Let's see if this is related to channels (e.g., some channels not allowing users to like videos, perhaps?).

In [12]:
videos[videos['likeCount'].isna()]['channelTitle'].value_counts()

BBC News Hindi                                      5
ExxonMobil                                          1
Intergovernmental Panel on Climate Change (IPCC)    1
Climate Change Committee                            1
Act of Knowledge                                    1
UNTV News and Rescue                                1
YALI Network                                        1
Name: channelTitle, dtype: int64

In [13]:
videos[videos['dislikeCount'].isna()]['channelTitle'].value_counts()

BBC News Hindi                                      5
ExxonMobil                                          1
Intergovernmental Panel on Climate Change (IPCC)    1
Climate Change Committee                            1
Act of Knowledge                                    1
UNTV News and Rescue                                1
YALI Network                                        1
Name: channelTitle, dtype: int64

OK, so here's the list of channels that have missing likes and dislikes. They are in any case the same. Now let's see if they appear in a list of channels that have likes.

In [14]:
videos[videos['viewCount'].isna()]['channelTitle'].value_counts()

YouTube Movies    1
Name: channelTitle, dtype: int64

And there is one channel that misses the number of views. It seems to be a movie. 

Let's check if the channels we have identified always have no likes or if they sometimes do.

First, let's make a list of all channels that do have likes and call it channels_with_likes.

In [15]:
channels_with_likes = videos[videos['likeCount'].isna()==False]['channelTitle'].unique().tolist()

In [16]:
channels_with_likes

['Full Frontal with Samantha Bee',
 'Info Viral',
 'KJ Singh',
 'BBC News Marathi',
 'The Coalition Radio Network',
 'Highest Truth',
 'TruthRadioShow',
 'KPBS',
 'CBC News',
 'Stuff',
 'LACityClerk',
 'UNTV News and Rescue',
 'Derek Knight',
 'CNBC Television',
 'California DWR',
 'US BREKING NEWS',
 'CNBC',
 'The News Forum',
 'Pat Gray Unleashed',
 'Make Peas Not Beef',
 'Global Soil Biodiversity Initiative',
 'Beril Sirmacek',
 'KOIN 6',
 'BBC News اردو',
 'ksbscotland',
 'PangeoHub',
 'GoNewsIndia',
 'FACE2FACE  C P JOHN  ',
 'SubC Imaging',
 'Sky News',
 'KSAT 12',
 'Ericsson',
 'Big Beardy',
 'Study IQ education',
 'SBS News',
 'BBC News',
 'Greenish planet',
 'SOMOY TV',
 'TomoNews US',
 'Scott Amyx',
 'Tony Heller',
 'Amazing Word Ministries',
 'NewsNation Now',
 'Associated Press',
 'KARE 11',
 'Denver7 – The Denver Channel',
 'Eagle News',
 'Allie Beth Stuckey',
 'The Independent',
 'Forbes Breaking News',
 'Washington Post',
 'Bloggingheads.tv',
 'Sir Jimmy Savile',
 'knowl

Using "in" operator we can check if an element is present in a list.

In [17]:
'BBC News Hindi' in channels_with_likes

False

In [18]:
'ExxonMobil' in channels_with_likes

False

This seems to be the case. But we cannot be very sure, so let's see how one of these video pages look like.

In [19]:
videos[videos['likeCount'].isna()]['videoId']

112    _SWdCTTq6GY
133    hYoG5jg2rPA
186    Ksdo25JdiJY
217    z149vLKn9d8
247    NnnjIGiEhR4
249    0Zs6SfN08o0
301    qbzqRW80w68
317    Ornz7UHgn8w
417    oxbreqk8DIQ
471    syQCKbjZlyo
474    HqHuJngkJkE
Name: videoId, dtype: object

Let me add the YouTube URL to some of these id's and see what's going on there:

* https://www.youtube.com/watch?v=NnnjIGiEhR4
* https://www.youtube.com/watch?v=hYoG5jg2rPA
* https://www.youtube.com/watch?v=syQCKbjZlyo
* https://www.youtube.com/watch?v=Ornz7UHgn8w


It seems that for all of them the like and dislike counters are turned off. But it does not necessarily mean that the missing values are always a specific number (e.g., a zero).


In [20]:
videos[videos['viewCount'].isna()]['videoId']

402    cU2HyoLfL38
Name: videoId, dtype: object

Let me add the YouTube URL to some of these id's and see what's going on there:

* https://www.youtube.com/watch?v=cU2HyoLfL38

Indeed, this type of video does not have views count.

Ultimately, I have generally two options with missing values:
* I can substitute them by another value (e.g., 0)
* I can drop them from the dataset

If I wanted to substitute them by another value, I would use the following command:

In [21]:
videos['dislikeCount_no_na'] = videos['dislikeCount'].fillna(0)

In [23]:
videos.isna().sum()

position                0
channelId               0
channelTitle            0
videoId                 0
publishedAt             0
publishedAtSQL          0
videoTitle              0
videoDescription       13
tags                   55
videoCategoryId         0
videoCategoryLabel      0
duration                0
durationSec             0
dimension               0
definition              0
caption                 0
thumbnail_maxres      117
licensedContent       114
viewCount               1
likeCount              11
dislikeCount           11
favoriteCount           0
commentCount           40
dislikeCount_no_na      0
dtype: int64

But it doesn't seem to be appropriate here, as the number of likes, dislikes and views for the identified videos is not being informed - we cannot see these values on YouTube either. So it is a limitation (that we need to acknowledge in our reporting), but most likely dropping these values is better. 

To do so, I can run the following command:

In [24]:
videos = videos.dropna(subset=['likeCount', 'dislikeCount', 'viewCount'])

In [25]:
videos.isna().sum()

position                0
channelId               0
channelTitle            0
videoId                 0
publishedAt             0
publishedAtSQL          0
videoTitle              0
videoDescription       13
tags                   51
videoCategoryId         0
videoCategoryLabel      0
duration                0
durationSec             0
dimension               0
definition              0
caption                 0
thumbnail_maxres      111
licensedContent       109
viewCount               0
likeCount               0
dislikeCount            0
favoriteCount           0
commentCount           35
dislikeCount_no_na      0
dtype: int64

In [26]:
len(videos)

488

### Checking the data types

It is also important to check if we have the data stored in the right format. Let's inspect it:

In [27]:
videos.dtypes

position                int64
channelId              object
channelTitle           object
videoId                object
publishedAt            object
publishedAtSQL         object
videoTitle             object
videoDescription       object
tags                   object
videoCategoryId         int64
videoCategoryLabel     object
duration               object
durationSec             int64
dimension              object
definition             object
caption                  bool
thumbnail_maxres       object
licensedContent       float64
viewCount             float64
likeCount             float64
dislikeCount          float64
favoriteCount           int64
commentCount          float64
dislikeCount_no_na    float64
dtype: object

From my key variables so far (likeCount, dislikeCount, viewCount, videoTitle, videoCategoryLabel), all looks OK. The numeric variables are in numeric form (int or float), and the text variables are in object form.

But the date variable (publishedAt) is stored as an object... and it should be a date.

In [28]:
videos['publishedAt'].head()

0    2021-09-16T05:59:45Z
1    2021-09-16T06:00:06Z
2    2021-09-16T05:59:46Z
3    2021-09-16T05:38:04Z
4    2021-09-16T04:58:32Z
Name: publishedAt, dtype: object

Yes, it looks like a date, but it is stored as an object. This is a problem, because I cannot filter the dataset by date.

In [29]:
videos['publishedAt'] = videos['publishedAt'].apply(pd.to_datetime)

In [30]:
videos['publishedAt'].head()

0   2021-09-16 05:59:45+00:00
1   2021-09-16 06:00:06+00:00
2   2021-09-16 05:59:46+00:00
3   2021-09-16 05:38:04+00:00
4   2021-09-16 04:58:32+00:00
Name: publishedAt, dtype: datetime64[ns, UTC]

In [31]:
videos.dtypes

position                            int64
channelId                          object
channelTitle                       object
videoId                            object
publishedAt           datetime64[ns, UTC]
publishedAtSQL                     object
videoTitle                         object
videoDescription                   object
tags                               object
videoCategoryId                     int64
videoCategoryLabel                 object
duration                           object
durationSec                         int64
dimension                          object
definition                         object
caption                              bool
thumbnail_maxres                   object
licensedContent                   float64
viewCount                         float64
likeCount                         float64
dislikeCount                      float64
favoriteCount                       int64
commentCount                      float64
dislikeCount_no_na                

Now we can for example check the videos published in 2019 and 2020, for example:

In [32]:
videos[videos['publishedAt'] >'2021-01-01']

,position,channelId,channelTitle,videoId,publishedAt,publishedAtSQL,videoTitle,videoDescription,tags,videoCategoryId,...,definition,caption,thumbnail_maxres,licensedContent,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,dislikeCount_no_na
0,1,UC18vz5hUUqxbGvym9ghtX_w,Full Frontal with Samantha Bee,xkgt1Avnvw0,2021-09-16 05:59:45+00:00,2021-09-16 05:59:45,Vivian Howard: The Perfect Recipe for Combatin...,Apocalyptic climate crisis got you down? Allow...,"Full Frontal with Samantha Bee,Full Frontal,Sa...",24,...,hd,False,https://i.ytimg.com/vi/xkgt1Avnvw0/maxresdefau...,1.0,796.0,88.0,2.0,0,9.0,2.0
1,2,UCJ6DCjlsOB8dwCrDbOLia6g,Info Viral,Sf-klthGrqA,2021-09-16 06:00:06+00:00,2021-09-16 06:00:06,"Global climate change!Cameroon under the snow,...",#GlobalClimateChange #CameroonUnderTheSnow #Sn...,"snow,snow fall,snow falling,snow fall in Camer...",26,...,hd,False,NaN,1.0,2.0,0.0,0.0,0,0.0,0.0
2,3,UC18vz5hUUqxbGvym9ghtX_w,Full Frontal with Samantha Bee,4Z0O1lJBU7g,2021-09-16 05:59:46+00:00,2021-09-16 05:59:46,Food Waste and Climate Change: How Your Leftov...,The recent devastating climate report left man...,"Full Frontal with Samantha Bee,Full Frontal,Sa...",24,...,hd,False,https://i.ytimg.com/vi/4Z0O1lJBU7g/maxresdefau...,1.0,1735.0,152.0,5.0,0,22.0,5.0
3,4,UCEfvFsy9qbzeKyASsDs0V-w,KJ Singh,KWoI9jTHHlk,2021-09-16 05:38:04+00:00,2021-09-16 05:38:04,Climate Change in Australia and Brexit #downun...,NaN,NaN,22,...,hd,False,https://i.ytimg.com/vi/KWoI9jTHHlk/maxresdefau...,NaN,0.0,0.0,0.0,0,0.0,0.0
4,5,UC7pluR6rB5KZIbN2IxamzxQ,BBC News Marathi,32y2dG2tLOg,2021-09-16 04:58:32+00:00,2021-09-16 04:58:32,Climate Change : 50c सेल्शिअस तापमानाच्या ठिका...,#ClimateChange #Temperature #Heat नायजेरिया दे...,"Global Warming,Warming,Hottest place in world,...",25,...,hd,False,NaN,1.0,5442.0,81.0,3.0,0,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,377,UCZYTClx2T1of7BRZ86-8fow,SciShow,1NBQZkySDY0,2021-01-22 22:53:49+00:00,2021-01-22 22:53:49,Oh No...Is Nature Going to Make Climate Change...,Photosynthesis plays a huge role in regulating...,"SciShow,science,Hank,Green,education,learn,cli...",27,...,hd,True,https://i.ytimg.com/vi/1NBQZkySDY0/maxresdefau...,1.0,164492.0,10263.0,272.0,0,1368.0,272.0
377,378,UCSrZ3UV4jOidv8ppoVuvW9Q,euronews,w3VUI3K2n5o,2021-01-21 18:08:31+00:00,2021-01-21 18:08:31,Did the coronavirus pandemic alter the course ...,2020 was a record-breaking year for the health...,"climate,climate change,global warming,climate ...",25,...,hd,False,https://i.ytimg.com/vi/w3VUI3K2n5o/maxresdefau...,1.0,63681.0,102.0,35.0,0,27.0,35.0
378,379,UC0p5jTq6Xx_DosDFxVXnWaQ,The Economist,7vOwjNTDwBE,2021-01-13 15:11:16+00:00,2021-01-13 15:11:16,How can business survive climate change? | The...,Climate change is about to upend the corporate...,"The Economist,Economist,Economist Films,Econom...",25,...,hd,True,https://i.ytimg.com/vi/7vOwjNTDwBE/maxresdefau...,1.0,246492.0,4445.0,191.0,0,518.0,191.0
379,380,UCMF296pHoCPUC_IxpYoG1Cg,UpLevel Green,ZwyEW_rDeqQ,2021-01-05 18:33:50+00:00,2021-01-05 18:33:50,"Jordan Peterson on Climate Change: ""8 Rules fo...",Jordan Peterson has some controversial thought...,"jordan peterson,climate change,greta thunberg,...",28,...,hd,False,https://i.ytimg.com/vi/ZwyEW_rDeqQ/maxresdefau...,NaN,1602.0,30.0,4.0,0,12.0,4.0


Or I can get the oldest and the latest date for the videos:

In [33]:
videos['publishedAt'].min()

Timestamp('2019-08-27 16:13:16+0000', tz='UTC')

In [34]:
videos['publishedAt'].max()

Timestamp('2021-09-16 06:00:06+0000', tz='UTC')

In [35]:
len(videos)

488

In [36]:
len(videos[videos['publishedAt'] >'2021-01-01'])

373

In [37]:
videos = videos[videos['publishedAt'] >'2021-01-01']

### Important!

Above we have used the ```.apply``` method to run a function in that column. Curious about other things you can do with it? Check out the notebook "UsefulFunctions" in the "UsefulScripts" folder.

OK, just to recap, this is the status of the variables that we need:
* Engagement: likeCount and dislikeCount are in the right type (int or float) and we fixed the missing values
* Sentiment: the videoTitle column is in the right type (object), but we don't have sentiment yet
* Category: we have the videoCategoryLabel, but we're not sure if it is really that informative yet
* PublishedAt: we corrected the data type, and managed to slice the dataframe correctly. Yes!

## Requesting sentiment analysis

As I don't have the language of the videos, I am going to use a Python module to automatically detect the language. It's available in the ```UsefulScripts``` folder, in the ```AdvancedModules``` notebook. If you want to use it, make sure to read the details in that notebook, as you may need to install a few things!

In [38]:
from langdetect import detect

In [39]:
def apply_langdetect(text):
    text = str(text)
    try:
        lang = detect(text)
    except:
        lang = 'error'
        
    return lang

In [40]:
videos['lang_title'] = videos['videoTitle'].apply(apply_langdetect)

In [41]:
videos['lang_title'].value_counts()

en    356
bn      7
ml      3
ro      2
mr      1
de      1
tl      1
fr      1
th      1
Name: lang_title, dtype: int64

In [42]:
videos[videos['lang_title']!='en'][['lang_title', 'videoTitle']]

,lang_title,videoTitle
4,mr,Climate Change : 50c सेल्शिअस तापमानाच्या ठिका...
19,de,BREAKING NEWS! BIDEN ADMITS SPEND $7 BILLION O...
31,ml,C P JOHN I ഇരുപത്തിആറാമത് കാലാവസ്ഥാ വ്യതിയാന സ...
41,bn,৩০ বছরে দেশের প্রায় দেড় কোটি মানুষ অভিবাসী হবে...
63,ml,പ്രളയം വരുന്നു ഗ്രീന്‍ലന്‍ഡില്‍ നിന്ന് / Green...
66,bn,চরম ভয়াবহতার সাক্ষী হতে যাচ্ছে পৃথিবী | Climat...
69,bn,'বাংলাদেশের ১৭ শতাংশ এলাকা পানিতে তলিয়ে যাবে'।...
72,bn,জলবায়ু পরিবর্তনের কারণে বিশ্বে গরম দিনের সংখ্য...
117,bn,পৃথিবীর ধ্বংস জলবায়ু পরিবর্তনেই | Climate Chan...
121,bn,তাহলে কি পৃথিবীর শেষ দিন ঘনিয়ে আসছে? | Climate...


The language detection module doesn't seem to be working all the time. But it did identify a video with a Russian title. So we can safely remove that video, and make a judgement call of what to do with the rest.

In [43]:
videos = videos[(videos['lang_title']!= 'bn') & (videos['lang_title']!= 'ml') & (videos['lang_title']!= 'tl') & (videos['lang_title']!= 'th')]

In [44]:
len(videos)

361

Now I can export it for sentiment analysis. When doing so, I will also need to change the column name - from videoTitle to text -, otherwise the sentiment analysis script won't know what to do.

In [45]:
sent_export = videos[['videoId', 'videoTitle']].rename(columns={'videoTitle': 'text'})

In [46]:
sent_export.head()

,videoId,text
0,xkgt1Avnvw0,Vivian Howard: The Perfect Recipe for Combatin...
1,Sf-klthGrqA,"Global climate change!Cameroon under the snow,..."
2,4Z0O1lJBU7g,Food Waste and Climate Change: How Your Leftov...
3,KWoI9jTHHlk,Climate Change in Australia and Brexit #downun...
4,32y2dG2tLOg,Climate Change : 50c सेल्शिअस तापमानाच्या ठिका...


In [47]:
sent_export.to_pickle('Sentiment/JoannaStrycharz_YouTubeClimateChange_EN.pkl')

Now I would usually upload this file to OneDrive (see links in the homepage of the General Repository) and have to wait one or two workdays until the analysis is complete. 

When it is complete, I will find the file in the ```SentimentAnalysisResults``` folder also on OneDrive. I have to download it and add to the same folder I am working on now.

In [48]:
sentiment = pd.read_pickle('Sentiment/JoannaStrycharz_YouTubeClimateChange_EN_completed.pkl')

In [49]:
sentiment.head()

,videoId,negative,positive,neutral
0,xkgt1Avnvw0,-1,1,0
1,Sf-klthGrqA,-1,2,1
2,4Z0O1lJBU7g,-1,1,0
3,KWoI9jTHHlk,-1,1,0
4,32y2dG2tLOg,-1,1,0


In [50]:
sentiment.dtypes

videoId     object
negative    object
positive    object
neutral     object
dtype: object

In [51]:
sentiment.isna().sum()

videoId     0
negative    0
positive    0
neutral     0
dtype: int64

### Important tip!

It seems I need to clean this variable (sentiment seems to be stored as an object). I will not show this in the video as this is part of the weekly challenge :-) 

*But I still need to merge the dataframes...*

## Merging the dataframes

Merging dataframes is an operation to bring the data of one dataframe into the other (or rather create a new dataframe). This is covered extensively in a video in the ```FAQs``` folder, and we will review it in class in a bit more detail (in DA3 and DA4).

Basically, we need to use the command ```.merge```. A few important tips:
* Make sure that you have one unique identifier (column) that is available in both dataframes
* Make sure that the unique identifier column has the same name in both dataframes
* Make sure that the unique identifier column is of the same data type in both dataframes

In our case, the unique identifier is videoId. Let's check the items above step by step:

In [52]:
videos.columns

Index(['position', 'channelId', 'channelTitle', 'videoId', 'publishedAt',
       'publishedAtSQL', 'videoTitle', 'videoDescription', 'tags',
       'videoCategoryId', 'videoCategoryLabel', 'duration', 'durationSec',
       'dimension', 'definition', 'caption', 'thumbnail_maxres',
       'licensedContent', 'viewCount', 'likeCount', 'dislikeCount',
       'favoriteCount', 'commentCount', 'dislikeCount_no_na', 'lang_title'],
      dtype='object')

In [53]:
sentiment.columns

Index(['videoId', 'negative', 'positive', 'neutral'], dtype='object')

In [54]:
videos.dtypes

position                            int64
channelId                          object
channelTitle                       object
videoId                            object
publishedAt           datetime64[ns, UTC]
publishedAtSQL                     object
videoTitle                         object
videoDescription                   object
tags                               object
videoCategoryId                     int64
videoCategoryLabel                 object
duration                           object
durationSec                         int64
dimension                          object
definition                         object
caption                              bool
thumbnail_maxres                   object
licensedContent                   float64
viewCount                         float64
likeCount                         float64
dislikeCount                      float64
favoriteCount                       int64
commentCount                      float64
dislikeCount_no_na                

In [55]:
sentiment.dtypes

videoId     object
negative    object
positive    object
neutral     object
dtype: object

OK, the column is available in both, and seems to be of the same data type. So I can merge.

In [56]:
len(videos)

361

In [57]:
len(sentiment)

360

In [58]:
videos.merge(sentiment, on='videoId')

,position,channelId,channelTitle,videoId,publishedAt,publishedAtSQL,videoTitle,videoDescription,tags,videoCategoryId,...,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,dislikeCount_no_na,lang_title,negative,positive,neutral
0,1,UC18vz5hUUqxbGvym9ghtX_w,Full Frontal with Samantha Bee,xkgt1Avnvw0,2021-09-16 05:59:45+00:00,2021-09-16 05:59:45,Vivian Howard: The Perfect Recipe for Combatin...,Apocalyptic climate crisis got you down? Allow...,"Full Frontal with Samantha Bee,Full Frontal,Sa...",24,...,796.0,88.0,2.0,0,9.0,2.0,en,-1,1,0
1,2,UCJ6DCjlsOB8dwCrDbOLia6g,Info Viral,Sf-klthGrqA,2021-09-16 06:00:06+00:00,2021-09-16 06:00:06,"Global climate change!Cameroon under the snow,...",#GlobalClimateChange #CameroonUnderTheSnow #Sn...,"snow,snow fall,snow falling,snow fall in Camer...",26,...,2.0,0.0,0.0,0,0.0,0.0,en,-1,2,1
2,3,UC18vz5hUUqxbGvym9ghtX_w,Full Frontal with Samantha Bee,4Z0O1lJBU7g,2021-09-16 05:59:46+00:00,2021-09-16 05:59:46,Food Waste and Climate Change: How Your Leftov...,The recent devastating climate report left man...,"Full Frontal with Samantha Bee,Full Frontal,Sa...",24,...,1735.0,152.0,5.0,0,22.0,5.0,en,-1,1,0
3,4,UCEfvFsy9qbzeKyASsDs0V-w,KJ Singh,KWoI9jTHHlk,2021-09-16 05:38:04+00:00,2021-09-16 05:38:04,Climate Change in Australia and Brexit #downun...,NaN,NaN,22,...,0.0,0.0,0.0,0,0.0,0.0,en,-1,1,0
4,5,UC7pluR6rB5KZIbN2IxamzxQ,BBC News Marathi,32y2dG2tLOg,2021-09-16 04:58:32+00:00,2021-09-16 04:58:32,Climate Change : 50c सेल्शिअस तापमानाच्या ठिका...,#ClimateChange #Temperature #Heat नायजेरिया दे...,"Global Warming,Warming,Hottest place in world,...",25,...,5442.0,81.0,3.0,0,5.0,3.0,mr,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,377,UCZYTClx2T1of7BRZ86-8fow,SciShow,1NBQZkySDY0,2021-01-22 22:53:49+00:00,2021-01-22 22:53:49,Oh No...Is Nature Going to Make Climate Change...,Photosynthesis plays a huge role in regulating...,"SciShow,science,Hank,Green,education,learn,cli...",27,...,164492.0,10263.0,272.0,0,1368.0,272.0,en,-4,1,-1
364,378,UCSrZ3UV4jOidv8ppoVuvW9Q,euronews,w3VUI3K2n5o,2021-01-21 18:08:31+00:00,2021-01-21 18:08:31,Did the coronavirus pandemic alter the course ...,2020 was a record-breaking year for the health...,"climate,climate change,global warming,climate ...",25,...,63681.0,102.0,35.0,0,27.0,35.0,en,-1,1,0
365,379,UC0p5jTq6Xx_DosDFxVXnWaQ,The Economist,7vOwjNTDwBE,2021-01-13 15:11:16+00:00,2021-01-13 15:11:16,How can business survive climate change? | The...,Climate change is about to upend the corporate...,"The Economist,Economist,Economist Films,Econom...",25,...,246492.0,4445.0,191.0,0,518.0,191.0,en,-1,1,0
366,380,UCMF296pHoCPUC_IxpYoG1Cg,UpLevel Green,ZwyEW_rDeqQ,2021-01-05 18:33:50+00:00,2021-01-05 18:33:50,"Jordan Peterson on Climate Change: ""8 Rules fo...",Jordan Peterson has some controversial thought...,"jordan peterson,climate change,greta thunberg,...",28,...,1602.0,30.0,4.0,0,12.0,4.0,en,-1,1,0


In [59]:
len(videos.merge(sentiment, on='videoId'))

368

Let's explore different types of merges a bit further.

In [60]:
len(videos.merge(sentiment, on='videoId', how='left'))

369

In [61]:
len(videos.merge(sentiment, on='videoId', how='right'))

368

In [62]:
len(videos.merge(sentiment, on='videoId', how='inner'))

368

In [63]:
len(videos.merge(sentiment, on='videoId', how='outer'))

369

In [64]:
videos_sent = videos.merge(sentiment, on='videoId')

We are almost there. Let's recap where we are:
* Engagement (OK): likeCount, dislikeCount and viewCount are in the right type (int or float) and we fixed the missing values
* Sentiment (OK-ish): we have the sentiment analysis results for the title, but they are in the wrong dtype. We won't fix it now - as you need to do it for the weekly challenge ;)
* Category: we have the videoCategoryLabel, but we're not sure if it is really that informative yet
* PublishedAt (OK): we corrected the data type, and managed to slice the dataframe correctly. Yes!

So we just need to work on the Category now.


## Video Categories

In [65]:
videos_sent['videoCategoryLabel'].value_counts()

News & Politics          220
Education                 44
People & Blogs            29
Entertainment             26
Science & Technology      21
Nonprofits & Activism     15
Comedy                     5
Travel & Events            3
Film & Animation           3
Howto & Style              1
Music                      1
Name: videoCategoryLabel, dtype: int64

We have waaaay too many categories here to make informative comparisons, and some of them are very small (1 video), while others have a lot of videos. While we that will almost always be the case for digital trace data, we can at least recategorize this a bit...

In [69]:
def recategorize(category):
    if category == 'News & Politics':
        return category
    if category == 'Education':
        return 'Education, Science and Technology'
    if category == 'Science & Technology':
        return 'Education, Science and Technology'
    if category == 'Nonprofits & Activism':
        return category
    else:
        return 'Other'
    

In [70]:
videos_sent['category'] = videos_sent['videoCategoryLabel'].apply(recategorize)

In [71]:
videos_sent['category'].value_counts()

News & Politics                      220
Other                                 68
Education, Science and Technology     65
Nonprofits & Activism                 15
Name: category, dtype: int64

This recategorization is not ideal, but at least we have four large(r) categories. One could argue that nonprofits is still too small. But for now we'll keep as is.

In [72]:
videos_sent.columns

Index(['position', 'channelId', 'channelTitle', 'videoId', 'publishedAt',
       'publishedAtSQL', 'videoTitle', 'videoDescription', 'tags',
       'videoCategoryId', 'videoCategoryLabel', 'duration', 'durationSec',
       'dimension', 'definition', 'caption', 'thumbnail_maxres',
       'licensedContent', 'viewCount', 'likeCount', 'dislikeCount',
       'favoriteCount', 'commentCount', 'dislikeCount_no_na', 'lang_title',
       'negative', 'positive', 'neutral', 'category'],
      dtype='object')

In [73]:
videos_sent.isna().sum()

position               0
channelId              0
channelTitle           0
videoId                0
publishedAt            0
publishedAtSQL         0
videoTitle             0
videoDescription      12
tags                  41
videoCategoryId        0
videoCategoryLabel     0
duration               0
durationSec            0
dimension              0
definition             0
caption                0
thumbnail_maxres      89
licensedContent       78
viewCount              0
likeCount              0
dislikeCount           0
favoriteCount          0
commentCount          30
dislikeCount_no_na     0
lang_title             0
negative               0
positive               0
neutral                0
category               0
dtype: int64

In [74]:
videos_sent.describe()

,position,videoCategoryId,durationSec,licensedContent,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,dislikeCount_no_na
count,368.000000,368.000000,368.000000,290.0,3.680000e+02,368.000000,368.000000,368.0,338.000000,368.000000
mean,196.095109,24.956522,638.904891,1.0,1.762282e+05,4337.567935,416.070652,0.0,1103.437870,416.070652
std,111.952326,2.846339,700.905226,0.0,7.072003e+05,19220.164569,2623.963882,0.0,4556.750099,2623.963882
min,1.000000,1.000000,15.000000,1.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000
25%,98.750000,25.000000,218.000000,1.0,2.245250e+03,54.750000,7.000000,0.0,24.000000,7.000000
50%,198.500000,25.000000,402.000000,1.0,1.459850e+04,241.000000,49.500000,0.0,137.500000,49.500000
75%,294.250000,25.000000,750.000000,1.0,7.982375e+04,1567.500000,156.500000,0.0,618.750000,156.500000
max,381.000000,29.000000,3585.000000,1.0,7.435424e+06,267054.000000,45290.000000,0.0,68194.000000,45290.000000


In [75]:
videos_sent['category'].value_counts()

News & Politics                      220
Other                                 68
Education, Science and Technology     65
Nonprofits & Activism                 15
Name: category, dtype: int64

In [76]:
videos_sent['category'].value_counts(normalize=True)

News & Politics                      0.597826
Other                                0.184783
Education, Science and Technology    0.176630
Nonprofits & Activism                0.040761
Name: category, dtype: float64

## We're done!

## But just because I am curious...

Let's see quickly how engagement varies...

In [77]:
videos_sent.groupby('neutral')[['likeCount', 'dislikeCount', 'viewCount']].describe().transpose()

neutral                       -1             0             1
likeCount    count  9.100000e+01  2.630000e+02  1.400000e+01
             mean   2.119143e+03  5.001376e+03  6.287214e+03
             std    5.234845e+03  2.217068e+04  1.689047e+04
             min    0.000000e+00  0.000000e+00  0.000000e+00
             25%    1.365000e+02  4.100000e+01  3.100000e+01
             50%    5.230000e+02  1.870000e+02  1.060000e+03
             75%    1.744500e+03  1.228000e+03  3.483500e+03
             max    4.007000e+04  2.670540e+05  6.450300e+04
dislikeCount count  9.100000e+01  2.630000e+02  1.400000e+01
             mean   2.218901e+02  4.856692e+02  3.707857e+02
             std    4.235332e+02  3.084362e+03  1.019189e+03
             min    0.000000e+00  0.000000e+00  0.000000e+00
             25%    2.500000e+01  4.000000e+00  1.250000e+01
             50%    7.200000e+01  4.000000e+01  5.500000e+01
             75%    1.955000e+02  1.435000e+02  1.602500e+02
             max    2.997000e+03  4.529000e+04  3.885000e+03
viewCount    count  9.100000e+01  2.630000e+02  1.400000e+01
             mean   1.415856e+05  1.802056e+05  3.266875e+05
             std    5.504898e+05  7.457688e+05  8.826503e+05
             min    2.200000e+01  0.000000e+00  2.000000e+00
             25%    5.981000e+03  1.582000e+03  1.740000e+03
             50%    3.105600e+04  8.506000e+03  3.394250e+04
             75%    9.768200e+04  6.687950e+04  7.741250e+04
             max    5.153641e+06  7.435424e+06  3.341806e+06

Pandas is using scientific notation because of other columns being too large. So let's change this.

In [78]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [79]:
videos_sent.groupby('neutral')[['likeCount', 'dislikeCount', 'viewCount']].describe().transpose()

neutral                     -1           0           1
likeCount    count      91.000     263.000      14.000
             mean     2119.143    5001.376    6287.214
             std      5234.845   22170.676   16890.473
             min         0.000       0.000       0.000
             25%       136.500      41.000      31.000
             50%       523.000     187.000    1060.000
             75%      1744.500    1228.000    3483.500
             max     40070.000  267054.000   64503.000
dislikeCount count      91.000     263.000      14.000
             mean      221.890     485.669     370.786
             std       423.533    3084.362    1019.189
             min         0.000       0.000       0.000
             25%        25.000       4.000      12.500
             50%        72.000      40.000      55.000
             75%       195.500     143.500     160.250
             max      2997.000   45290.000    3885.000
viewCount    count      91.000     263.000      14.000
             mean   141585.560  180205.570  326687.500
             std    550489.832  745768.750  882650.307
             min        22.000       0.000       2.000
             25%      5981.000    1582.000    1740.000
             50%     31056.000    8506.000   33942.500
             75%     97682.000   66879.500   77412.500
             max   5153641.000 7435424.000 3341806.000

In [80]:
videos_sent.groupby('category')[['likeCount', 'dislikeCount', 'viewCount']].mean().transpose()

category,"Education, Science and Technology",News & Politics,Nonprofits & Activism,Other
likeCount,12317.154,2257.755,2305.200,3887.147
dislikeCount,1028.554,329.214,92.867,182.912
viewCount,447470.969,113570.941,120068.533,132054.824


In [81]:
videos_sent.groupby('category')[['likeCount', 'dislikeCount', 'viewCount']].std().transpose()

category,"Education, Science and Technology",News & Politics,Nonprofits & Activism,Other
likeCount,41581.233,7042.328,5546.039,11237.791
dislikeCount,5650.599,1392.316,150.259,637.465
viewCount,1379156.918,441305.367,202106.345,436535.035


To make it more readable, you can use transpose (columns become rows, and vice-versa)

In [82]:
videos_sent.to_pickle('videos_sent.pkl')